In [20]:
from collections.abc import Generator
from operator import itemgetter
from pathlib import Path
from typing import Any, TypedDict

import librosa
import numpy as np
import polars as pl
import torch
from faster_whisper import WhisperModel
from intervaltree import Interval, IntervalTree
from resemblyzer import VoiceEncoder
from silero_vad import get_speech_timestamps, load_silero_vad
from sklearn.cluster import AgglomerativeClustering


# Диаризация аудио

Задача была сведена к задачи кластеризации. Распространенные модели в виде pyannote, nemo titanet, wespeaker, Gemini DF Resnet используемые во фреймворке sherpa_onnx дают неудовлетворительные результаты для речи на русском языке. Поэтому было принято решение собрать алгоритм самостоятельно используя следующие технологии:
* Silero VAD - для решения задачи обнаружения голосовой активности
* resemblyzer - для формирования embedding-ов для отрезков аудио
* AgglomerativeClustering - из библиотеки sklearn для кластеризации

Также для получения объективной оценки качеству решения задачи был разработан бенчмарк, размечен небольшой датасет (около 20 минут аудио). В качестве метрики была выбрана ARI. Были получены следующие результаты:
|name|micro_ari|macro_ari|
|----|---------|---------|
|resemblyzer + silero_vad|0,525|0,507|
|pyannote-segmentation-3-0 + 3dspeaker_speech_campplus_sv_zh_en_16k-common_advanced|0,174|0,113|
|pyannote-segmentation-3-0 + voxblink2_samresnet34_ft|0,167|0,113|
|pyannote-segmentation-3-0 + voxblink2_samresnet100_ft|0,421|0,404|
|pyannote-segmentation-3-0 + voxceleb_gemini_dfresnet114_LM|0,276|0,179|
|pyannote-segmentation-3-0 + wespeaker_en_voxceleb_resnet293_LM|0,007|0,003|
|pyannote-segmentation-3-0 + nemo_en_titanet_large|0,175|0,113|

Ознакомится с бэнчмарком можно в папке benchmark/main.py

In [3]:
class Segment(TypedDict):
    start: float
    end: float
    label: str

In [4]:
def resemblyzer_silero_vad(filepath: Path, n_clusters: int | None = 2) -> Generator[Segment, Any, Any]:
    encoder = VoiceEncoder("cpu")
    model = load_silero_vad(onnx=True)
    wav_numpy, _ = librosa.load(filepath, sr=16000, mono=True)
    wav = torch.from_numpy(wav_numpy).float()
    speech_timestamps = get_speech_timestamps(
        wav,
        model,
        return_seconds=True,
    )

    embeddings = []
    valid_segments = []

    for start, end in map(itemgetter("start", "end"), speech_timestamps):
        start_sample = int(start * 16000)
        end_sample = int(end * 16000)

        segment = wav_numpy[start_sample:end_sample]

        if len(segment) < 6400:
            segment = np.pad(segment, (0, 6400 - len(segment)))

        try:
            emb = encoder.embed_utterance(segment)
            embeddings.append(emb)
            valid_segments.append({"start": start, "end": end})
        except Exception as e:
            continue

    if embeddings:
        embeddings_array = np.array(embeddings)

        if len(embeddings_array) > 1:
            clustering = AgglomerativeClustering(
                n_clusters=n_clusters,
                distance_threshold=0.90 if n_clusters is None else None,
            )
            labels = clustering.fit_predict(embeddings_array)

            for i, seg in enumerate(valid_segments):
                if seg["end"] - seg["start"] > 0.3:
                    yield {
                        "start": seg["start"],
                        "end": seg["end"],
                        "label": f"SPEAKER_{labels[i]:02d}",
                    }

## Пример работы пайплайна

In [12]:
segments = list(resemblyzer_silero_vad(Path("./audio/3827885e-public_1.wav")))

Loaded the voice encoder model on cpu in 0.00 seconds.


In [13]:
segments[0:10]

[{'start': 0.2, 'end': 9.8, 'label': 'SPEAKER_00'},
 {'start': 10.1, 'end': 17.9, 'label': 'SPEAKER_00'},
 {'start': 19.0, 'end': 20.6, 'label': 'SPEAKER_00'},
 {'start': 21.5, 'end': 22.4, 'label': 'SPEAKER_00'},
 {'start': 22.5, 'end': 30.8, 'label': 'SPEAKER_01'},
 {'start': 31.1, 'end': 37.9, 'label': 'SPEAKER_01'},
 {'start': 38.1, 'end': 41.4, 'label': 'SPEAKER_01'},
 {'start': 42.8, 'end': 44.8, 'label': 'SPEAKER_00'},
 {'start': 47.9, 'end': 51.0, 'label': 'SPEAKER_00'},
 {'start': 51.3, 'end': 56.4, 'label': 'SPEAKER_01'}]

# Распознавание речи

Эта задача уже очень хорошо решена компанией OpenAI и ее моделью Whisper. Для распознавания используется эта модель. Запуск происходит с помощью библиотеки faster_whisper.

In [7]:
def transcibe(filepath: Path):
    model = WhisperModel("medium", device="auto")

    segments, _ = model.transcribe(filepath, word_timestamps=True, vad_filter=True)  # type: ignore  # noqa: PGH003

    for segment in segments:
        if segment.words is not None:
            for word in segment.words:
                yield (word.start, word.end, word.word)

## Пример работы пайплайна

In [11]:
words = list(transcibe(Path("./audio/3827885e-public_1.wav")))

In [14]:
words[0:10]

[(np.float64(0.0), np.float64(0.24), ' У'),
 (np.float64(0.24), np.float64(0.42), ' нас,'),
 (np.float64(0.42), np.float64(0.62), ' к'),
 (np.float64(0.62), np.float64(0.96), ' сожалению,'),
 (np.float64(1.18), np.float64(1.8), ' произошли'),
 (np.float64(1.8), np.float64(2.22), ' некоторые'),
 (np.float64(2.22), np.float64(3.18), ' неполадки.'),
 (np.float64(3.26), np.float64(3.68), ' Повторю'),
 (np.float64(3.68), np.float64(4.14), ' вопрос.'),
 (np.float64(4.5), np.float64(4.66), ' Вот')]

# Сопоставление слов каждому из говорящих

Для сопоставления слов используется структура данных IntervalTree c оптимальной скоростью вставки и поиском пересекающихся интервалов. (На основе AVL tree все в среднем за log(n)). Сначала записываются интервалы для каждого слова, а затем выполняется поиск слов которые попадают в заданный сегмент.

In [ ]:
tree = IntervalTree()

In [22]:
for word in words:
    timestamp_to = word[1]
    if word[0] == timestamp_to:
        timestamp_to += 0.001
    tree.add(Interval(word[0], timestamp_to, data=word[2]))

In [26]:
for segment in segments[0:10]:
    results = tree.overlap(segment["start"], segment["end"])
    print(segment["label"], "".join(tuple(iv.data for iv in sorted(results))))

SPEAKER_00  У нас, к сожалению, произошли некоторые неполадки. Повторю вопрос. Вот мы с тобой ранее обсуждали, что при изучении иностранных языков возникает такой вот
SPEAKER_00  вот интересный момент, что ты начинаешь их путать. Вот, можешь показать и рассказать какие-нибудь ситуации, которые у тебя были?
SPEAKER_00  Или в целом ощущениях?
SPEAKER_00  Быть-было.
SPEAKER_01  Ты сначала думаешь о какой-то предложении на русском, потом оно переводится у тебя в клавиатурный английский, ты планируешь сказать это
SPEAKER_01  на английском, начинаешь говорить на китайском, оно смешивается с английским, и ты говоришь на двух языках одновременно,
SPEAKER_01  но мозг не всегда сразу понимает, что что-то не так.
SPEAKER_00  Или ты сформулировал предложение... Это ты имеешь в виду?
SPEAKER_00  Это ты имеешь в виду, когда говоришь с иностранцем на английском, да?
SPEAKER_01  Когда ты говоришь... Или с преподавателем. Это очень забавно, когда ты говоришь с китайцем,
